In [1]:
!pip install emoji
import emoji as emoji
import numpy as np
import pandas as pd

  Using cached emoji-1.4.2.tar.gz (184 kB)
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186457 sha256=c8733e777b940575cde005b000485822181dbe29ce7c278ab496430d5e5ccb2f
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\71\4d\3c\cada364d4ea0026deee7208dee1e61bcebd20aa2ae5dc154ba
Successfully built emoji


In [5]:
train = pd.read_csv('D:/VS/Coding-Blocks-ML/RNN/dataset/train_emoji.csv', header=None)
test = pd.read_csv('D:/VS/Coding-Blocks-ML/RNN/dataset/test_emoji.csv', header=None)

In [6]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [7]:
emoji_dictionary = {"0": "\u2764\uFE0F", #:heart:
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",}

In [8]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [9]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [10]:
data = train.values
for i in range(10):
    print(data[i][0], emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [12]:
from tensorflow.keras.utils import to_categorical

In [13]:
XT = train[0]
Xt = test[0]
YT = to_categorical(train[1])
Yt = to_categorical(test[1])

print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [15]:
embeddings = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')

        embeddings[word] = coeffs

In [18]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [19]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

<ipython-input-18-4b9aff165303>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [20]:
print(emb_Xt.shape)
print(emb_XT.shape)

(56, 10, 50)
(132, 10, 50)


In [21]:
from keras.layers import *
from keras.models import Sequential

In [22]:
model = Sequential()
model.add(LSTM(64, input_shape=(10, 50), return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64, input_shape=(10, 50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(emb_XT, YT, batch_size=32, epochs=40, shuffle=True, validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 45s 801ms/step - loss: 1.6056 - acc: 0.2608 - val_loss: 1.5745 - val_acc: 0.2143
Epoch 2/40
4/4 [==============================] - 0s 34ms/step - loss: 1.5194 - acc: 0.3525 - val_loss: 1.5878 - val_acc: 0.0714
Epoch 3/40
4/4 [==============================] - 0s 28ms/step - loss: 1.4822 - acc: 0.4354 - val_loss: 1.5723 - val_acc: 0.1429
Epoch 4/40
4/4 [==============================] - 0s 28ms/step - loss: 1.4334 - acc: 0.4497 - val_loss: 1.5264 - val_acc: 0.3571
Epoch 5/40
4/4 [==============================] - 0s 29ms/step - loss: 1.3894 - acc: 0.4763 - val_loss: 1.4737 - val_acc: 0.3571
Epoch 6/40
4/4 [==============================] - 0s 29ms/step - loss: 1.2911 - acc: 0.5636 - val_loss: 1.4160 - val_acc: 0.3571
Epoch 7/40
4/4 [==============================] - 0s 33ms/step - loss: 1.2043 - acc: 0.5542 - val_loss: 1.4072 - val_acc: 0.2857
Epoch 8/40
4/4 [==============================] - 0s 28ms/step - loss: 1.1009 - acc: 0.5532 - v

In [24]:
model.evaluate(emb_Xt, Yt)

2/2 [==============================] - 0s 11ms/step - loss: 1.8758 - acc: 0.6071


[1.8757517337799072, 0.6071428656578064]

In [25]:
pred = model.predict_classes(emb_Xt)

C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [26]:
for i in range(30):
    print(" ".join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😁
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
